# Title

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * a
  

### Default input of a step

In a process-oriented workflow, **the default input of a step is the output of its previous step**. For example, in the following workflow, the first step (step `10`) produces output `a.txt`, which becomes the input of the second step (step `20`).

In [2]:
%run

[10]
output: 'a.txt'
_output.touch()

[20]
print(f'Input of step {step_name} is {step_input}')

,default,Workflow ID 763b03b0293e70ac,Index #1,completed Ran for < 5 seconds


Input of step 20 is a.txt


## Further reading

* 